In [1]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
cards = []

url_base = 'https://pt.petitchef.com'
url = url_base + '/receitas/rapida'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
}

try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)

soup = BeautifulSoup(html, 'html.parser')

In [3]:
pages = int(soup.find('div', class_="pages").get_text().split()[-1])

for i in range(pages):
    url_page = url_base + '/receitas/rapida-page-' + str(i + 1)

    try:
        req = Request(url_page, headers = headers)
        response = urlopen(req)
        html = response.read().decode('utf-8')
        
    except HTTPError as e:
        print(e.status, e.reason)
        
    except URLError as e:
        print(e.reason)

    soup = BeautifulSoup(html, 'html.parser')
    
    # receitas = soup.find('div', {"id": "recipe-list"}).findAll('div', class_="item clearfix")
    receitas = soup.findAll('div', class_="ingredients")

    for item in receitas:
        card = {}
        receita = item.findPrevious('div', {'class': 'item clearfix'})

        card['nome'] = receita.find('h2', {'class': 'ir-title'}).find('a').getText()
        card['link_receita'] = url_base + receita.find('h2', {'class': 'ir-title'}).find('a')['href']

        try:
            card['url_imagem'] = url_base + receita.find('div', {'class': 'i-left'}).find('img')['src']
            # urlretrieve(card['url_imagem'], './output/img/' + imagem_url.split('/')[-1])
        except:
            card['url_imagem'] = 'Indisponível'

        try:
            card['avaliacao'] = receita.find('div', {'class': 'ir-vote'}).find('i')['title']
        except:
            card['avaliacao'] = 'Indisponível'
        
        try:
            card['comentarios'] = receita.find('div', {'class': 'ir-vote'}).find('i', {'class': 'fal fa-comments fa-fw'}).getText().strip()
        except:
            card['comentarios'] = 'Indisponível'
        
        try:
            card['marcacoes'] = receita.find('div', {'class': 'ir-vote'}).find('i', {'class': 'fal fa-bookmark fa-fw'}).getText().strip()
        except:
            card['marcacoes'] = 'Indisponível'

        try:
            card['tipo'] = receita.find('div', {'class': 'prop'}).find('i', {'class': 'fas fa-utensils'}).findPrevious().getText().strip()
        except:
            card['tipo'] = 'Indisponível'

        try:
            card['dificuldade'] = receita.find('div', {'class': 'prop'}).find('i', {'class': 'fas fa-signal'}).findPrevious().getText().strip()
        except:
            card['dificuldade'] = 'Indisponível'

        try:
            card['tempo_preparo'] = receita.find('div', {'class': 'prop'}).find('i', {'class': 'fas fa-clock'}).findPrevious().getText().strip()
        except:
            card['tempo_preparo'] = 'Indisponível'

        try:
            card['calorias'] = receita.find('div', {'class': 'prop'}).find('i', {'class': 'fas fa-fire'}).findPrevious().getText().strip()
        except:
            card['calorias'] = 'Indisponível'

        try:
            card['tempo_cozedura'] = receita.find('div', {'class': 'prop'}).find('i', {'class': 'fas fa-scale-balanced'}).findPrevious().getText().strip()
        except:
            card['tempo_cozedura'] = 'Indisponível'

        try:
            card['resumo_ingredientes'] = receita.find('div', {'class': 'ingredients'}).getText()
        except:
            card['resumo_ingredientes'] = 'Indisponível'
        
        cards.append(card)

dataset = pd.DataFrame(cards)
dataset.to_csv('./output/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')
dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset

,nome,link_receita,url_imagem,avaliacao,comentarios,marcacoes,tipo,dificuldade,tempo_preparo,calorias,tempo_cozedura,resumo_ingredientes
0,"Macarrão cremoso com curgete (abobrinha), fáci...",https://pt.petitchef.com/receitas/prato-princi...,https://pt.petitchef.com/imgupl/feed-data/1579...,3.00/5 (1 votos),Indisponível,Indisponível,Prato Principal,Muito Fácil,13 min,12 min,Indisponível,Ingredientes: 190 g de macarrão 100 gr de quei...
1,"Mousse de damasco, rápido, sem cozimento e com...",https://pt.petitchef.com/receitas/sobremesa/mo...,https://pt.petitchef.com/imgupl/feed-data/1580...,Indisponível,Indisponível,Indisponível,Sobremesa,Muito Fácil,3 h 12 m,Indisponível,65 kcal,Ingredientes: 150 gr damascos (sem caroços) 1 ...
2,"Biscoito folhado de canela, fácil e rápido a p...",https://pt.petitchef.com/receitas/lanche/bisco...,https://pt.petitchef.com/imgupl/feed-data/1579...,Indisponível,Indisponível,Indisponível,Lanche,Muito Fácil,30 min,Indisponível,75 kcal,Ingredientes: 1 massa folhada retangular 80 gr...
3,Ramen de frango: uma deliciosa sopa oriental f...,https://pt.petitchef.com/receitas/prato-princi...,https://pt.petitchef.com/imgupl/feed-data/1579...,3.67/5 (3 votos),Indisponível,Indisponível,Prato Principal,Muito Fácil,45 min,35 min,Indisponível,Ingredientes: 2 files de frango 200 gr de mass...
4,"Creme de couve, rápida e deliciosa",https://pt.petitchef.com/receitas/entrada/crem...,https://pt.petitchef.com/imgupl/feed-data/1579...,3.00/5 (1 votos),Indisponível,Indisponível,Entrada,Muito Fácil,22 min,Indisponível,143 kcal,Ingredientes: 1/2 couve kale (couve frisada ou...
...,...,...,...,...,...,...,...,...,...,...,...,...
366,Empadão de arroz com atum,https://pt.petitchef.com/receitas/prato-princi...,https://pt.petitchef.com/imgupl/recipe-type/md...,4.00/5 (1 votos),Indisponível,,Prato Principal,Muito Fácil,15 min,20 min,Indisponível,Ingredientes: 3 dentes alho 1 malagueta parti...
367,Omelete de courgette e queijo com salada à cató,https://pt.petitchef.com/receitas/prato-princi...,https://pt.petitchef.com/imgupl/recipe-type/md...,3.00/5 (1 votos),Indisponível,,Prato Principal,Muito Fácil,20 min,20 min,Indisponível,Ingredientes: 1 tomate chucha rijinho 1 queij...
368,Salada de bulgur e salmão com molho pesto,https://pt.petitchef.com/receitas/entrada/sala...,https://pt.petitchef.com/imgupl/recipe-type/md...,2.00/5 (1 votos),Indisponível,,Entrada,Muito Fácil,15 min,1 min,Indisponível,Ingredientes: Meia posta de salmão grelhado 3...
369,Oatmeal chocolate chip muffins (muffins de ave...,https://pt.petitchef.com/receitas/outro/oatmea...,https://pt.petitchef.com/imgupl/recipe-type/md...,Indisponível,Indisponível,Indisponível,Outro,Muito Fácil,45 min,Indisponível,567 kcal,Ingredientes: 160 g de farinha 90 g de aveia ...
